In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd gdrive/MyDrive/PHD/SemEval2023

/content/gdrive/MyDrive/PHD/SemEval2023


In [ ]:
# import pandas as pd
# unsupervised_data_gab = pd.read_csv("Data/starting_ki/gab_1M_unlabelled.csv")
# unsupervised_data_reddit = pd.read_csv("Data/starting_ki/reddit_1M_unlabelled.csv")

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Initialize the stopwords
stoplist = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [ ]:
import string

def preprocess(sent):
  sent = sent.lower()
  words = nltk.word_tokenize(sent)

  # Remove Stop words
  words = [w for w in words if w not in stoplist]

  # Remove Punctuation
  words = [w for w in words if w not in string.punctuation]
  
  # Lematization
  
  words = [lemmatizer.lemmatize(w) for w in words]
  return words


# Pointwise mutual information (PMI)

<img src="https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEhPFTkoN4lp4MA1do7rOHhOx_v5DjyyoELu4pKJ52Bh0XldLX5uJetgocwm9PoCi0Sx-dr4bdDBm2BD5q9rOB950cOKFEBv-L8pbJ6BQuoVkWYCJ7mbOrqElQQtjIVGq9AYsYw46sMmO7E7lSf7-_nJ1KURR7X8cY6VWqaJRL9bnzg6JnJKctPIoe0iuQ/s1600/PPMI.PNG">

Read: https://nocodefunctions.com/blog/pmi-tf-idf/


In [ ]:
import pandas as pd

In [ ]:
from collections import Counter

In [ ]:
target_column = "label_category"
data = pd.read_csv("Data/starting_ki/train_all_tasks.csv")
data = data[data[target_column]!="none"]

In [ ]:
texts = data["text"].values
words = [preprocess(s) for s in texts]
labels = data[target_column].values

In [ ]:
if target_column == "label_category":
  label_values = [
      '1. threats, plans to harm and incitement',
      '2. derogation',
      '3. animosity',
      '4. prejudiced discussions',
  ]

#   label_map = {
#       0: '1. threats, plans to harm and incitement',
#       1: '2. derogation', 
#       2: '3. animosity', 
#       3: '4. prejudiced discussions',
#       '1. threats, plans to harm and incitement':0,
#       '2. derogation':1,
#       '3. animosity':2,
#       '4. prejudiced discussions':3,
#   }
elif target_column == "label_vector":
  label_values = [
      '1.1 threats of harm',
      '1.2 incitement and encouragement of harm',
      '2.1 descriptive attacks',
      '2.2 aggressive and emotive attacks',
      '2.3 dehumanising attacks & overt sexual objectification',
      '3.1 casual use of gendered slurs, profanities, and insults',
      '3.2 immutable gender differences and gender stereotypes',
      '3.3 backhanded gendered compliments',
      '3.4 condescending explanations or unwelcome advice',
      '4.1 supporting mistreatment of individual women',
      '4.2 supporting systemic discrimination against women as a group',
  ]

#   label_map = {
#       0: '1.1 threats of harm',
#       1: '1.2 incitement and encouragement of harm',
#       2: '2.1 descriptive attacks',
#       3: '2.2 aggressive and emotive attacks',
#       4: '2.3 dehumanising attacks & overt sexual objectification',
#       5: '3.1 casual use of gendered slurs, profanities, and insults',
#       6: '3.2 immutable gender differences and gender stereotypes',
#       7: '3.3 backhanded gendered compliments',
#       8: '3.4 condescending explanations or unwelcome advice',
#       9: '4.1 supporting mistreatment of individual women',
#       10: '4.2 supporting systemic discrimination against women as a group',
#       '1.1 threats of harm': 0,
#       '1.2 incitement and encouragement of harm': 1,
#       '2.1 descriptive attacks': 2,
#       '2.2 aggressive and emotive attacks': 3,
#       '2.3 dehumanising attacks & overt sexual objectification': 4,
#       '3.1 casual use of gendered slurs, profanities, and insults': 5,
#       '3.2 immutable gender differences and gender stereotypes': 6,
#       '3.3 backhanded gendered compliments': 7,
#       '3.4 condescending explanations or unwelcome advice': 8,
#       '4.1 supporting mistreatment of individual women': 9,
#       '4.2 supporting systemic discrimination against women as a group': 10,
#   }
# else:
#   raise Exception("Unknown label column")

In [ ]:
import torch
from sklearn.model_selection import train_test_split

train_words, test_words, train_labels, test_labels = train_test_split(words, labels, random_state=42, test_size=0.2)

In [ ]:
Cx = Counter()
Cy = Counter()
Cxy = Counter()

In [ ]:
for sent in train_words:
  Cx.update(sent)

for idx, sent in enumerate(train_words):
  for w in sent:
    Cxy[(labels[idx], w)] += 1

Cy.update(labels)

In [ ]:
sum(Cxy.values()), sum(Cx.values())

(38503, 38503)

In [ ]:
vocabs = Cx.keys()

In [ ]:
import math

Nx = sum(Cx.values())
Ny = sum(Cy.values())
N = Nx + Ny

PMI = {}
for label in label_values:
  PMI[label] = {}

for label in label_values:
  Py = Cy[label]/N
  for w in vocabs:
    Px = Cx[w]/N
    Pxy = Cxy[(label, w)]/N

    delta = 1e-10
    PMI[label][w] = max(math.log2((Pxy+delta)/(Px*Py)), 0)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline


def select_lexicons(label_values, vocabs, PMI, q=0.9):
  lexicons = {}
  for label in label_values:
    lexicons[label] = set()
    
    pmi_values = PMI[label].values()

    pmi_values = [p for p in pmi_values if p > 0]
    threshold = np.quantile(pmi_values, q)
    for w in vocabs:
      if PMI[label][w] >= threshold:
        lexicons[label].add(w)
  return lexicons

In [ ]:
lexicons = select_lexicons(label_values, vocabs, PMI, q=0.75)

In [ ]:
def save_lexicons(filepath, obj):
  with open(filepath, 'w', encoding="utf-8") as fin:
    for w in obj:
      fin.write(str(w)+"\n")

for idx, label in enumerate(label_values):
  print("Label", label)
  print("#Lexicons", len(lexicons[label]))

  save_lexicons(f"./Results/PMI_lexicon_{idx+1}.txt", lexicons[label])
  # for l in lexicons[label][0:50]:
  #   print(l)
  print("")

Label 1. threats, plans to harm and incitement
#Lexicons 411

Label 2. derogation
#Lexicons 2035

Label 3. animosity
#Lexicons 1539

Label 4. prejudiced discussions
#Lexicons 571



## Evaluation

In [ ]:
Cy

Counter({'2. derogation': 1590,
         '1. threats, plans to harm and incitement': 310,
         '3. animosity': 1165,
         '4. prejudiced discussions': 333})

In [ ]:
# test_words, train_labels, test_labels

In [ ]:
from sklearn.metrics import f1_score

def predict(word, lexicons):
  cnt = {}
  for label in label_values:
    cnt[label] = 0

  for w in word:
    for label in label_values:
      if w in lexicons[label]:
        cnt[label] += 1

  
  if sum(cnt.values())==0:
    return None
  
  return max(cnt.items(), key=lambda k: k[1])[0]

lexicons = select_lexicons(label_values, vocabs, PMI, q=0.75)

y_pred = []
y_test = []
non = 0
for word, label in zip(test_words, test_labels):
  pred = predict(word, lexicons)
  if pred is None:
    non += 1
    continue

  y_pred.append(pred)
  y_test.append(label)

print("F1:", f1_score(y_test, y_pred, average='macro'))
print("Skip:", non/len(test_words))

F1: 0.24188623890648447
Skip: 0.45294117647058824


In [ ]:
# for i in range(1, 100):
  
#   lexicons = select_lexicons(label_values, vocabs, PMI, q=i/100)

#   y_pred = []
#   y_test = []
#   non = 0
#   for word, label in zip(test_words, test_labels):
#     pred = predict(word, lexicons)
#     if pred is None:
#       non += 1
#       continue

#     y_pred.append(pred)
#     y_test.append(label)

#   f1 = f1_score(y_test, y_pred, average='macro')
#   print("Hyperparameter:", i/100, f1, non/len(test_words))